# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
driver = webdriver.Chrome()

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [3]:
driver.get("https://www.tdlr.texas.gov/tools_search")

In [4]:
number = driver.find_element_by_id("mcrdata")

In [5]:
number.send_keys("006179570C")
search = driver.find_element_by_id ("submit3")
search.click()

In [7]:
cells = driver.find_elements_by_tag_name('td')
name = cells[5].text
owner = cells[7].text
phone_number = cells[9].text
license_status = cells[12].text
listing=driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text
physical_address=listing.split(" ")[19:]

# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [172]:
driver.get("https://www.tdlr.texas.gov/tools_search")
number = driver.find_element_by_id("mcrdata")
number.send_keys("006179570C")
search = driver.find_element_by_id ("submit3")
search.click()
cells = driver.find_elements_by_tag_name('td')
name = cells[5].text
owner = cells[7].text
phone_number = cells[9].text
license_status = cells[12].text
listing = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text
physical_address=listing.split(" ")[19:]


# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [173]:
import pandas as pd
df=pd.read_csv('/Users/nickospi/Desktop/trucks-subset.csv')   

## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [174]:
pd.set_option('display.max_colwidth', -1)
df['TDLR Number']

0    006507931C
1    006179570C
2    006502097C
Name: TDLR Number, dtype: object

In [175]:
def make_url (n):
    return url+n

In [176]:
for number in df['TDLR Number']:
    n = number
    url = 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber='


In [177]:
df['TDLR Number'].apply(make_url)

0    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
Name: TDLR Number, dtype: object

### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [178]:
df['url'] = df.apply(make_url) 
df

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [1]:
def get_tow_info(row):
    driver.get(row['url'])
    cells = driver.find_elements_by_tag_name('td')
    name = cells[5].text.split()[1:]
    owner = cells[7].text
    phone_number = cells[9].text
    license_status = cells[12].text
    listing = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text
    physical_address=listing.split(" ")[19:]   
    return pd.Series({
    'Name': name,
    'Owner': owner,
    'Phone_Number': phone_number,
    'License_Status': license_status,
    'Physical_address':physical_address
})

## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [204]:
driver = webdriver.Chrome()
new_df = df.apply(get_tow_info, axis=1).join(df)
new_df.head()

,Name,Owner,Phone_Number,License_Status,Physical_address,TDLR Number,url
0,"[AUGUSTUS, E, SMITH]",Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Status: Active,"[N, MAIN, ST\nBONHAM,, TX., 75418]",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,"[B.D., SMITH, TOWING]",Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Status: Expired,"[76179\n\nPhysical:\n13619, BRETT, JACKSON, RD.\nFORT, WORTH,, TX., 76179]",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,"[BARRY, MICHAEL, SMITH]",Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Status: Active,"[W, CEMETERY, RD\nCANYON,, TX., 79015]",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [205]:
new_df.to_csv("tow-trucks-extended.csv", index = False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [206]:
pd.read_csv('/Users/nickospi/Desktop/tow-trucks-extended.csv')

,Name,Owner,Phone_Number,License_Status,Physical_address,TDLR Number,url
0,"['AUGUSTUS', 'E', 'SMITH']",Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Status: Active,"['N', 'MAIN', 'ST\nBONHAM,', 'TX.', '75418']",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,"['B.D.', 'SMITH', 'TOWING']",Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Status: Expired,"['76179\n\nPhysical:\n13619', 'BRETT', 'JACKSON', 'RD.\nFORT', 'WORTH,', 'TX.', '76179']",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,"['BARRY', 'MICHAEL', 'SMITH']",Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Status: Active,"['W', 'CEMETERY', 'RD\nCANYON,', 'TX.', '79015']",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [207]:
full_df=pd.read_csv('/Users/nickospi/Desktop/tow-trucks.csv')   

In [208]:
full_df['TDLR Number'].apply(make_url)

0     https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1     https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2     https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3     https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4     https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF
5     https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006448786C
6     https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0648444VSF
7     https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0651667VSF
8     https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006017767C
9     https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C
10    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006518521C
11    https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006

In [209]:
full_df['url'] = full_df.apply(make_url) 
full_df

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF
5,006448786C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006448786C
6,0648444VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0648444VSF
7,0651667VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0651667VSF
8,006017767C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006017767C
9,006495492C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C


In [210]:
def get_tow_info(row):
    driver.get(row['url'])
    cells = driver.find_elements_by_tag_name('td')
    name = cells[5].text.split()[1:]
    owner = cells[7].text
    phone_number = cells[9].text
    license_status = cells[12].text
    listing = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text
    physical_address=listing.split(" ")[19:]   
    return pd.Series({
    'Name': name,
    'Owner': owner,
    'Phone_Number': phone_number,
    'License_Status': license_status,
    'Physical_address':physical_address
})

In [211]:
driver = webdriver.Chrome()
full_df = full_df.apply(get_tow_info, axis=1).join(full_df)
full_df.head()

,Name,Owner,Phone_Number,License_Status,Physical_address,TDLR Number,url
0,"[AUGUSTUS, E, SMITH]",Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Status: Active,"[N, MAIN, ST\nBONHAM,, TX., 75418]",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,"[B.D., SMITH, TOWING]",Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Status: Expired,"[76179\n\nPhysical:\n13619, BRETT, JACKSON, RD.\nFORT, WORTH,, TX., 76179]",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,"[BARRY, MICHAEL, SMITH]",Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Status: Active,"[W, CEMETERY, RD\nCANYON,, TX., 79015]",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,"[HEATH, SMITH]",Owner/Officer: HEATH A SMITH / OWNER,Phone: 940-552-0687,Status: Expired,"[ST\nVERNON,, TX., 76384]",006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,"[HEATH, SMITH]",Owner/Officer: HEATH A SMITH / OWNER,Phone: 9405520687,Status: Expired,[],0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF


In [215]:
full_df.to_csv("tow-trucks-extended.csv", index = False)